
# Finding Profitable Mobile App Niches in Google Play and iOS App Store


The purpose of this anaylsis is to identify mobile app niches that have high profitability potential.  Our client develops 'Free' apps and earns revenue from in-app ads.  For this business model to be successful (ad revenue), the apps must attract a lot of users.

Their development process is:
1. Identify a profitable app niche on the Google Play store. (The aim of this project)
1. Create an MVP Android app, release, and promote.
1. If the Google Play app is successful, create an iOS version.

We will use app marketplace data to make data driven recommendations for the development team. 


## Exploring the Data


As of the third quarter of 2022, there are 3.5 million apps available on the Google Play store and 1.6 million apps on Apple's App Store ([Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)).

These datasets do not appear to be freely available. However, we were able to locate two free datsets that are samples.  These will allow us to explore the data and determine what insights (if any) can be gained at no cost.  The two datasets that we will use in this analysis are:

* [This sample](https://www.kaggle.com/datasets/lava18/google-play-store-apps) of about 10k Google Play Store apps that was web scraped.
* [This sample](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) of 7,200 Apple Store apps that were obtained through the iTunes API.

In [1]:
# Read in the csv files
from csv import reader

open_apple = open('my_datasets/AppAdRev/AppleStore.csv', encoding='utf8')
open_google = open('my_datasets/AppAdRev/googleplaystore.csv', encoding='utf8')

read_apple = reader(open_apple)
read_google = reader(open_google)

apple_apps = list(read_apple)
google_apps = list(read_google)

For convenience, we have written a helper function `explore_data()` to make the dataset easier to read.

In [2]:
# Helper function to view data
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Let's print the head of each list of lists.

In [3]:
# Explore the data format
print('Apple Apps')
explore_data(apple_apps, 0, 5, True)
print('\n\n')
print('Google Apps')
explore_data(google_apps, 0, 5, True)

Apple Apps
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7198
Number of columns: 16



Google Apps
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', '

## Cleaning the Data

In the discussion on Kaggle for the Google Play dataset, it was agreed that there is a row of corrupted data.  Below we validate the bad data row and remove it from the dataset.

In [4]:
print(google_apps[10473])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [5]:
# Delete the bad row 
del google_apps[10473]

In [6]:
print(len(google_apps))

10841


In the discussion on Kaggle for the Google Play dataset, it was observed that there are duplicate entries for the same app names.  The app names in this dataset do not function as primary keys.  Here we check for duplicates.

In [7]:
# Check the Google data for duplicate entries by app name
unique_apps = []
dupe_apps = []
for row in google_apps[1:]:
    app_name = row[0]
    if app_name in unique_apps:
        dupe_apps.append(app_name)
    else:
        unique_apps.append(app_name)

In [8]:
# These are the duplicate rows
print('Total duplicates: ' + str(len(dupe_apps)))
print(dupe_apps[:5])

Total duplicates: 1181
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


Let's take a closer look at some duplicate entries.

In [9]:
# Print all the data for the first 5 rows of duplicates
for app in dupe_apps[:5]:
    for row in google_apps[1:]:
        if app == row[0]:
            print(row)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000+', 'Fr

There doesn't appear to be much difference (if any) in the duplicate rows.  In the discussion, it is thought that the duplicates were a snapshot at different times and we should therefore look at the 'rating_count_total' column and keep the row with the highest review count.  This seems like a reasonable assumption so we'll keep the row with the highest number of reviews.

In [10]:
# Loop through each app and creat a dict with app as key and the highest review count as val 
reviews_max = {}
for row in google_apps[1:]:
    name = row[0]
    n_reviews = float(row[3])
    
    # Check if the name is in reviews_max first so reviews_max[name] won't throw a key error
    if name in reviews_max and n_reviews > reviews_max[name]:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In [11]:
print(len(reviews_max))

9659


Remove the duplicate rows

In [12]:
"""
Many of the duplicate rows have review counts that are identical so we must keep track 
once we have added the app to the clean list using the app name as a primary key.
"""
android_clean = []
already_added = []

for row in google_apps[1:]:
    name = row[0]
    n_reviews = float(row[3])
    
    if name not in already_added and n_reviews == reviews_max[name]:
        android_clean.append(row)
        already_added.append(name)


In [13]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13



## Removing Non-English Apps


Our client only develops apps in English but the data includes apps in other languages.  We'll check the unicode character of each letter in app names using `ord()`. ASCII printable characters are codes 32 - 127.  To allow for some flexibility like emojis, we'll allow a maximum of three non-ASCII characters in an app name.

In [14]:
# Check if the characters in the app name are ASCII
def is_english(a_string):
    over_127 = 0
    for letter in a_string:
        if ord(letter) > 127:
            over_127 += 1
    if over_127 > 3:
        return False
    else:
        return True        

In [15]:
# Make sure it works as intended
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


In [16]:
# Remove non-english ios apps
ios_eng = []
for row in apple_apps[1:]:     # row 0 is the header
    app_name = row[1]          # iOS app name is index 1
    if is_english(app_name):
        ios_eng.append(row)

# Remove non-english android apps
android_clean_eng = []
for row in android_clean:      # there is no header row
    app_name = row[0]          # android app name is index 0
    if is_english(app_name):
        android_clean_eng.append(row)

In [17]:
explore_data(ios_eng, 0, 3, True)
explore_data(android_clean_eng, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies


## Isolate the Free Apps


Our client only builds free apps and the revenue is generated by ad revenue.  Both datasets (Android and iOS) contain both paid and free apps.  Here we will extract the free apps to analyse.

In [18]:
# In the iOS dataset, free apps have a 'price' column that equals '0.0' (string).
free_ios_apps = []
for row in ios_eng:
    price = row[4]
    if price == '0.0':
        free_ios_apps.append(row)

# In the Android dataset, free apps are catergorized in the 'type' column that equals 'Free' (string).
free_android_apps = []
for row in android_clean_eng:
    app_type = row[6]
    if app_type == 'Free':
        free_android_apps.append(row)

In [19]:
print(len(free_android_apps))
print(len(free_ios_apps))

8863
3222



## Explore the Free App Genres by Frequency


Let's consider apps by Genre and create a frequency table.  The iOS genre is labeled `prime_genre` (index 11 or -5) and the Android data has two options: `Genres` (index 9) and `Category` (index  1).  If free market participants are using resources to create more apps of certain generes, maybe it's because they are the more profitable niches.

In [20]:
# Dataset is a list of lists, index is an integer
def freq_table(dataset, index):
    # total counts
    f_table = {}
    for row in dataset:
        key = row[index]
        if key in f_table:
            f_table[key] += 1
        else:
            f_table[key] = 1
    
    # as percentages
    table_perc = {}
    app_count = len(dataset)
    for key in f_table:
        table_perc[key] = round(((f_table[key] / app_count) * 100), 2)
        
    return table_perc
    

In [21]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


### Frequency Table (`prime_genre`, iOS, %)


In [22]:
display_table(free_ios_apps, 11)

Games : 58.16
Entertainment : 7.88
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.51
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.33
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12



### Frequency Table (`Genres`, Android, %)


In [23]:
display_table(free_android_apps, 9)

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.9
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Games : 0.17
Entertainment;Music & Video : 0.17
Casual;B


### Frequency Table (`category`, Andriod, %)


In [24]:
display_table(free_android_apps, 1)

FAMILY : 18.9
GAME : 9.73
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6



## Explore the Free App Genres by Installs



Now that we have considered the supply side, let's consider the demand side by analysing the most downloaded apps by genre and category.  The iOS dataset does not have installs as a column value.  Therefore we will use the average user ratings as a proxy.

In [25]:
ios_genre_freq_table = freq_table(free_ios_apps, 11)

In [26]:
ios_genre_avg_ratings = []

for genre in ios_genre_freq_table:
    total = 0
    len_genre = 0
    
    for row in free_ios_apps:
        # prime_genre index is 9 or -5
        app_genre = row[-5]
        
        if genre == app_genre:
            # rating_count_total is index 5
            total += int(row[5])
            len_genre += 1
    # list of tuples with sort index as first element        
    ios_genre_avg_ratings.append((round(total / len_genre), genre))
            
sorted(ios_genre_avg_ratings, reverse=True)

[(86090, 'Navigation'),
 (74942, 'Reference'),
 (71548, 'Social Networking'),
 (57327, 'Music'),
 (52280, 'Weather'),
 (39758, 'Book'),
 (33334, 'Food & Drink'),
 (31468, 'Finance'),
 (28442, 'Photo & Video'),
 (28244, 'Travel'),
 (26920, 'Shopping'),
 (23298, 'Health & Fitness'),
 (23009, 'Sports'),
 (22789, 'Games'),
 (21248, 'News'),
 (21028, 'Productivity'),
 (18684, 'Utilities'),
 (16486, 'Lifestyle'),
 (14030, 'Entertainment'),
 (7491, 'Business'),
 (7004, 'Education'),
 (4004, 'Catalogs'),
 (612, 'Medical')]

Let's check the installs on Android.

In [27]:
display_table(free_android_apps, 5)

1,000,000+ : 15.73
100,000+ : 11.55
10,000,000+ : 10.55
10,000+ : 10.2
1,000+ : 8.39
100+ : 6.92
5,000,000+ : 6.83
500,000+ : 5.56
50,000+ : 4.77
5,000+ : 4.51
10+ : 3.54
500+ : 3.25
50,000,000+ : 2.3
100,000,000+ : 2.13
50+ : 1.92
5+ : 0.79
1+ : 0.51
500,000,000+ : 0.27
1,000,000,000+ : 0.23
0+ : 0.05


In [28]:
android_cat = freq_table(free_android_apps, 1)

In [29]:
android_genres = []
for category in android_cat:
    total = 0
    len_category = 0
    
    for row in free_android_apps:
        category_app = row[1]
        if category_app == category:
            # The installs string has '+' and ',' chars that need to be removed before using int()
            total += int(row[5].replace("+", "").replace(",", ""))
            len_category += 1
            
    android_genres.append((round((total / len_category)), category))
    

In [30]:
sorted(android_genres, reverse=True)

[(38456119, 'COMMUNICATION'),
 (24727872, 'VIDEO_PLAYERS'),
 (23253652, 'SOCIAL'),
 (17840110, 'PHOTOGRAPHY'),
 (16787331, 'PRODUCTIVITY'),
 (15588016, 'GAME'),
 (13984078, 'TRAVEL_AND_LOCAL'),
 (11640706, 'ENTERTAINMENT'),
 (10801391, 'TOOLS'),
 (9549178, 'NEWS_AND_MAGAZINES'),
 (8767812, 'BOOKS_AND_REFERENCE'),
 (7036877, 'SHOPPING'),
 (5201483, 'PERSONALIZATION'),
 (5074486, 'WEATHER'),
 (4188822, 'HEALTH_AND_FITNESS'),
 (4056942, 'MAPS_AND_NAVIGATION'),
 (3697848, 'FAMILY'),
 (3638640, 'SPORTS'),
 (1986335, 'ART_AND_DESIGN'),
 (1924898, 'FOOD_AND_DRINK'),
 (1833495, 'EDUCATION'),
 (1712290, 'BUSINESS'),
 (1437816, 'LIFESTYLE'),
 (1387692, 'FINANCE'),
 (1331541, 'HOUSE_AND_HOME'),
 (854029, 'DATING'),
 (817657, 'COMICS'),
 (647318, 'AUTO_AND_VEHICLES'),
 (638504, 'LIBRARIES_AND_DEMO'),
 (542604, 'PARENTING'),
 (513152, 'BEAUTY'),
 (253542, 'EVENTS'),
 (120551, 'MEDICAL')]


## Conclusion


By far, the most common app genre available on iOS App Store is 'Games' (58%).  The next closest genre was 'Entertainment' at 8%.  However, based on installs, the top threee app genres are: 'Navigation', 'Reference', 'Social Networking'.  

Navigation is dominated by utilities like Google Maps and Waze.  This probably wouldn't be a good niche to compete in.

After further review, we recommend exploration with the development team on the following niches to see if a competitve advantage could be reasonably achieved: 

* Games
* Weather 
* Photo and Video
* Tools  